In [2]:
import h5py
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
pick = pd.read_pickle('drugpredictor2/data/05_model_input/input_table.pickle')

In [4]:
pick_val = pd.read_pickle('drugpredictor2/data/05_model_input/input_table_val.pickle')

In [11]:
val_dataset_mock = pd.read_pickle('drugpredictor2/data/05_model_input/input_table_val_mock.pickle')

In [12]:
val_dataset_mock

,SMILES,Molecular Weight,H-Bond Donors,H-Bond Acceptors,LogP,RuleFive,RDKit_Molecule,Morgan2FP
15,CN1CCC(Oc2c(C(=O)Nc3ccccc3)nnn2-c2ccccc2)CC1,377.448,1,6,2.99260,1,<rdkit.Chem.rdchem.Mol object at 0x000001EC298...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
23,COc1ccc(C=NNC(=O)c2n[nH][nH]c2=S)cc1,277.309,3,5,1.23979,1,<rdkit.Chem.rdchem.Mol object at 0x000001EC0C5...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
19,O=C(Nc1ccccc1)c1nnn(-c2ccccc2)c1OCc1ccccc1,370.412,1,5,4.09860,1,<rdkit.Chem.rdchem.Mol object at 0x000001EC0C5...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,CNC(=O)c1n[nH]n(N=Cc2ccc(OC)cc2)c1=O,275.268,2,6,-0.17820,1,<rdkit.Chem.rdchem.Mol object at 0x000001EC298...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [3]:
with h5py.File('drugpredictor2/data/05_model_input/model_input.h5', 'w') as f:
    # Create a dataset with chunking enabled
    # Adjust chunks to be a manageable size for your memory
    f.create_dataset('my_data', data=pick, chunks=(1000, 100), compression='gzip')
print("Array saved to large_array.h5 with chunking.")

ValueError: Chunk shape must not be greater than data shape in any dimension. (1000, 100) is not compatible with (1324896, 8)

In [6]:
pick_val.columns

Index(['SMILES', 'Molecular Weight', 'H-Bond Donors', 'H-Bond Acceptors',
       'LogP', 'RuleFive', 'RDKit_Molecule', 'Morgan2FP'],
      dtype='object')

In [7]:
pick.columns

NameError: name 'pick' is not defined

In [73]:
# Folder containing CSV files
folder_path = os.path.join('drugpredictor2', 'data','01_raw', 'csv')

# List to store DataFrames
dataframes = []

# Iterate over files in the folder
for file in os.listdir(folder_path):
    # Check if the file is a CSV
    if file.endswith('.csv'):
        file_path = os.path.join(folder_path, file)
        
        # Read CSV into DataFrame and append to list
        df = pd.read_csv(file_path)
        dataframes.append(df)

# Concatenate all DataFrames in the list
combined_df = pd.concat(dataframes, ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv(folder_path+'/combined_output.csv', index=False)

print("CSV files concatenated successfully.")

CSV files concatenated successfully.


In [105]:
""" ESTAMOS VALIDANDO ESTA"""
def train_test_split_column(model_input: pd.DataFrame, columns: dict, label: dict) -> tuple:
    """
    Function that makes a train/test split of a selected column of a pandas dataframe. Y values are taken from the label column.
    Args:
      input_pickle: input dataframe.
      column: name of the column selected as X values.
      label: name of the column selected as y values (target).
    Output: train/test tuple.
    """
    X = model_input[columns]
    y = model_input[label]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    return X_train, X_test, y_train, y_test

In [106]:
model_input = pd.read_pickle('drugpredictor2/data/05_model_input/input_table.pickle')
model_input

,SMILES,Molecular Weight,H-Bond Donors,H-Bond Acceptors,LogP,RuleFive,RDKit_Molecule,Morgan2FP
0,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C,203.238,0,4,-1.2357,0,<rdkit.Chem.rdchem.Mol object at 0x00000273395...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,CC(=O)OC(CC(=O)O)C[N+](C)(C)C,204.246,1,3,0.0990,1,<rdkit.Chem.rdchem.Mol object at 0x00000273395...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,O=C(O)C1=CC=CC(O)C1O,156.137,3,3,-0.7110,0,<rdkit.Chem.rdchem.Mol object at 0x00000273395...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,CC(O)CN,75.111,2,2,-0.6741,1,<rdkit.Chem.rdchem.Mol object at 0x00000273395...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,NCC(=O)COP(=O)(O)O,169.073,3,4,-1.3765,0,<rdkit.Chem.rdchem.Mol object at 0x00000273395...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,O=[N+]([O-])c1ccc(Cl)c([N+](=O)[O-])c1,202.553,0,4,2.1564,0,<rdkit.Chem.rdchem.Mol object at 0x00000273395...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,CCn1cnc2c(N)ncnc21,163.184,1,5,0.4284,1,<rdkit.Chem.rdchem.Mol object at 0x00000273395...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,CCC(C)(O)C(O)C(=O)O,148.158,3,3,-0.4071,1,<rdkit.Chem.rdchem.Mol object at 0x00000273395...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8,O=P(O)(O)OC1C(O)C(O)C(O)C(O)C1O,260.135,7,7,-3.7176,0,<rdkit.Chem.rdchem.Mol object at 0x00000273395...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [107]:
label = 'RuleFive'
X_column = 'Morgan2FP'

In [130]:
def reshape_input(tuple_split: tuple) -> tuple:
    """
    Function that reshapes the input data (X_train, X_test) to fit a 1D CNN model and determines both the number of classes \
        from the label data and the shape of the reshaped data.
    Input: train/test split tuple.
    Output: tuple with the data necessary to run the CNN model.
    """
    X_train, X_test, y_train, y_test = tuple_split
    print('X_train ', X_train, 'SHAPE: ', X_train.shape)
    print('X_test ', X_test)
    print('y_train ', y_train)
    # Transform arrays into lists
    X_train_array = np.array(list(X_train))
   
    X_test_array = np.array(list(X_test))
    
    # Determine number of classes
    #print(y_train)
    n_classes = len(np.unique(y_train))
    # Reshape
    reshaped_X_train= X_train_array.reshape((X_train_array.shape[0], X_train_array.shape[1], 1))
    reshaped_X_test= X_test_array.reshape((X_test_array.shape[0], X_test_array.shape[1], 1))
    # Determine in_shape
    in_shape = reshaped_X_train.shape[1:]
    #print(reshaped_X_train, reshaped_X_test, in_shape, n_classes)
    return reshaped_X_train, reshaped_X_test, in_shape, n_classes

In [133]:
def current_node(model_input: pd.DataFrame, x_column, label) -> tuple:
    splits = train_test_split_column(model_input, x_column, label)
    reshaped_X_train, reshaped_X_test, in_shape, n_classes = reshape_input(splits)
    print(reshaped_X_train, reshaped_X_train.shape)
    print(reshaped_X_test)
    print(in_shape)
    print(n_classes)

In [134]:
current_node(model_input,X_column,label)

X_train  0    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
8    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
6    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: Morgan2FP, dtype: object SHAPE:  (6,)
X_test  7    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
5    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: Morgan2FP, dtype: object
y_train  0    0
8    0
2    0
4    0
3    1
6    1
Name: RuleFive, dtype: int64
[[[0]
  [1]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [1]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]] (6, 2048, 1)
[[[0]
  [1]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [1]
